In [21]:
%pip install redis



Note: you may need to restart the kernel to use updated packages.


In [1]:

import redis

r = redis.Redis(host='localhost', port=6379, db=0)

In [2]:
from counter import get_last_counter

bank_id = f"bank_{get_last_counter()}"
bank_id

'bank_42'

In [46]:
r.keys()

[b'faiss_index:v1::bank_40',
 b'faiss_index:v1::bank_39',
 b'faiss_index:v1::bank_38',
 b'memory_banks:v1::bank_38',
 b'memory_banks:v1::bank_39',
 b'memory_banks:v1::bank_40']

In [47]:
r.get(f"memory_banks:v1::{bank_id}")

b'{"identifier":"bank_40","provider_resource_id":"bank_40","provider_id":"faiss","type":"memory_bank","memory_bank_type":"vector","embedding_model":"all-MiniLM-L6-v2","chunk_size_in_tokens":512,"overlap_size_in_tokens":null}'

In [6]:
r.info()

{'redis_version': '7.4.1',
 'redis_git_sha1': 0,
 'redis_git_dirty': 0,
 'redis_build_id': '7fd3520e9c14a41b',
 'redis_mode': 'standalone',
 'os': 'Linux 6.8.0-49-generic x86_64',
 'arch_bits': 64,
 'monotonic_clock': 'POSIX clock_gettime',
 'multiplexing_api': 'epoll',
 'atomicvar_api': 'c11-builtin',
 'gcc_version': '11.4.0',
 'process_id': 184895,
 'process_supervised': 'systemd',
 'run_id': 'a2cdc0a08c4d4eb4607a4792fb4e962e225ee8a6',
 'tcp_port': 6379,
 'server_time_usec': 1733637871975374,
 'uptime_in_seconds': 4488,
 'uptime_in_days': 0,
 'hz': 10,
 'configured_hz': 10,
 'lru_clock': 5584623,
 'executable': '/usr/bin/redis-server',
 'config_file': '/etc/redis/redis.conf',
 'io_threads_active': 0,
 'listener0': {'name': 'tcp', 'bind': '-::1', 'port': 6379},
 'connected_clients': 3,
 'cluster_connections': 0,
 'maxclients': 10000,
 'client_recent_max_input_buffer': 8,
 'client_recent_max_output_buffer': 0,
 'blocked_clients': 0,
 'tracking_clients': 0,
 'pubsub_clients': 0,
 'watch

In [3]:
from json import loads
from IPython.display import display

index = loads(r.get(f"faiss_index:v1::{bank_id}"))
# Write index to JSON file
import json
with open('faiss_index.json', 'w') as f:
    json.dump(index, f, indent=2)
display(index.keys())
display(len(index['id_by_index']))

dict_keys(['id_by_index', 'chunk_by_index', 'faiss_index'])

175

In [23]:
import os
from llama_stack_client import LlamaStackClient

# Test run
client = LlamaStackClient(
    base_url="http://localhost:5001",
)

response = client.memory.query(
    bank_id=bank_id,
    query=["Hello World"],
    params={"max_chunks": 10},
)

display(response)

for chunk, score in zip(response.chunks, response.scores):
    print(chunk)
    print(score)


QueryDocumentsResponse(chunks=[Chunk(content=" puts the BMW in drive and floors\n     it out of the parking lot toward the REAR DRIVEWAY.\n\n     As he rounds the corner... Choco is on a balcony with a huge\n     television OVER HIS HEAD. Chuckle slams on the breaks.\n\n                          CHUCKIE\n                Fuck.\n\n     Choco HURLS THE TELEVISION AND IT PLUNGES THROUGH THE\n     WINDSHIELD INTO THE FRONT SEAT OF THE CAR.\n\n     Choco jumps fifteen feet off the balcony onto the hood of the\n     BMW, lifts Chuckie out of the driver's seat and launches him\n     like a two pound bag of sugar.\n\n                          CHOCO\n                AAAAHHHHHHHHH!!!!!\n\n     Chuckie travels a good twenty feet and lands on his head.\n\n     INT. CREW WINNEBAGO -- NEXT\n\n     Weiss and Kimmie watch the monitors in awe.\n\n                          WEISS\n                Good lord.\n\n     EXT. 15 NORTH FREEWAY -- NIGHT\n\n     HELICOPTER SHOT: The Winnebagos make their way east 

Chunk(content=" puts the BMW in drive and floors\n     it out of the parking lot toward the REAR DRIVEWAY.\n\n     As he rounds the corner... Choco is on a balcony with a huge\n     television OVER HIS HEAD. Chuckle slams on the breaks.\n\n                          CHUCKIE\n                Fuck.\n\n     Choco HURLS THE TELEVISION AND IT PLUNGES THROUGH THE\n     WINDSHIELD INTO THE FRONT SEAT OF THE CAR.\n\n     Choco jumps fifteen feet off the balcony onto the hood of the\n     BMW, lifts Chuckie out of the driver's seat and launches him\n     like a two pound bag of sugar.\n\n                          CHOCO\n                AAAAHHHHHHHHH!!!!!\n\n     Chuckie travels a good twenty feet and lands on his head.\n\n     INT. CREW WINNEBAGO -- NEXT\n\n     Weiss and Kimmie watch the monitors in awe.\n\n                          WEISS\n                Good lord.\n\n     EXT. 15 NORTH FREEWAY -- NIGHT\n\n     HELICOPTER SHOT: The Winnebagos make their way east on the 15\n     FREEWAY.\n\n   

In [24]:
import pandas as pd

datasets = {
    "narrativeqa": "data/narrativeqa.parquet",
}

df = pd.read_parquet(datasets["narrativeqa"])
df['context_length'] = df['context'].apply(lambda x: len(x))
df.describe()

,context_length
count,50.000000
mean,217480.920000
std,58733.594823
min,110830.000000
25%,186267.000000
50%,215859.500000
75%,240608.500000
max,417011.000000


In [25]:
doc_ids = []
questions = []
for _, row in df.iterrows():
    for question in row["questions"]:
        questions.append(question)
        doc_ids.append(_)

print(len(questions))
print(len(doc_ids))
display(questions[:5])
doc_ids[:40]

1490
1490


['Who planned the robbery that was being investigated when Domino Harvey was arrested?',
 'Which career was Domino in before she became a bounty hunter?',
 "What did Lateesha plan to use to pay for her granddaughter's medical procedure?",
 'How did Claremont know there would be $10 Million in the armored car?',
 'Who were the FBI initially led to believe would commit the robbery?']

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1]

In [26]:
from tqdm import tqdm

top_k = 10

results = []

for i in tqdm(range(0, len(questions))):
    response = client.memory.query(
        bank_id=bank_id,
        query=[questions[i]],
        params={"max_chunks": top_k},
    )
    res = [int(chunk.document_id) for chunk in response.chunks]
    results.append(res)

results

 18%|██████████████                                                               | 272/1490 [00:08<01:57, 10.40it/s]

100%|████████████████████████████████████████████████████████████████████████████| 1490/1490 [01:49<00:00, 13.59it/s]


[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0,

In [28]:
import numpy as np

targets = np.array(doc_ids)
display(targets.shape)
targets = np.expand_dims(targets, axis=1)
display(targets.shape)
targets = targets == res
display(targets.shape)
targets = targets.astype(float)
display(targets)

(1490,)

(1490, 1)

(1490, 10)

array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [29]:
top_k = 10

res = np.array(results)
res = res[:, :top_k]
targets = targets[:, :top_k]
display(targets.shape)
display(res.shape)

(1490, 10)

(1490, 10)

In [30]:
temp = np.sum(targets, axis=1)
temp = np.divide(temp, top_k)
precision = np.sum(temp) / len(temp)
precision

np.float64(0.020134228187919462)

In [31]:
temp = np.logical_or.reduce(targets, axis=1)
temp = temp.astype(float)
recall = np.sum(temp) / len(temp)
recall

np.float64(0.020134228187919462)

In [32]:
ranks = np.tile(np.arange(1, top_k + 1), (len(questions), 1)).astype(float)
temp = np.copy(targets)
for i in range(len(temp)):
    first_true = True
    for j in range(len(temp[i])):
        if temp[i,j]:
            if first_true:
                first_true = False
            else:
                temp[i,j] = False
reciprocal_ranks = temp / ranks
reciprocal_ranks = np.sum(reciprocal_ranks) / len(temp)
reciprocal_ranks = np.mean(reciprocal_ranks)
reciprocal_ranks

np.float64(0.020134228187919462)